In [2]:
# !pip install deep_translator
# !pip install langdetect
# !pip install langid

In [3]:
import pandas as pd
import re
from deep_translator import GoogleTranslator
import pickle

In [4]:
with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)



In [5]:
def book_rating(ISBN):
    # Get the title of the book, author and language with the given ISBN
    book_title = books_data.loc[books_data['ISBN'] == ISBN, 'Book-Title'].iloc[0]
    book_author = books_data.loc[books_data['ISBN'] == ISBN, 'Book-Author'].iloc[0]
    book_language = books_data.loc[books_data['ISBN'] == ISBN, 'Language'].iloc[0]
    
    if book_language != 'en':
        english_title = GoogleTranslator(source='auto', target='en').translate(book_title)
    else:
        english_title = book_title
    
    # Get all the books that the author wrote
    authors_books = books_data.loc[books_data['Book-Author'] == book_author, ['ISBN', 'Book-Title', 'Language']]
    authors_books['English-Title'] = None
    
    # Get the english title for all the books
    authors_books['English-Title'].loc[authors_books['Language'] == 'en'] = authors_books.loc[authors_books['Language'] == 'en', 'Book-Title']
    authors_books['English-Title'].loc[authors_books['Language'] != 'en'] = authors_books.loc[authors_books['Language'] != 'en', 'Book-Title'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))   
    
    # Filter the DataFrame for books with titles whose leftmost 30 characters are the same as translated_title
    matched_books = authors_books.loc[authors_books['English-Title'].str[:30] == english_title[:30]]
                                                                                                                                                     
    books_rating = ratings_data.loc[ratings_data['ISBN'].isin(matched_books['ISBN']), 'Book-Rating'].mean()
    
    return books_rating

In [7]:
def recommendation(BookIsLiked, BookTitle, BookAuthor):
    # Find books matching the given title and author
    filtered_books = books_data[(books_data['Book-Title'].str.contains(BookTitle)) & (books_data['Book-Author'].str.contains(BookAuthor))]
    
    # Get ratings of users who rated similar books
    users_ratings_books = pd.merge(users_data, ratings_data, on='User-ID')
    users_ratings_books = pd.merge(users_ratings_books, filtered_books[['ISBN']], on='ISBN')
    users_ratings_books = users_ratings_books[users_ratings_books['Book-Rating'] > BookIsLiked]
    
    # Find users who liked similar books
    lord_readers = users_ratings_books['User-ID'].unique()
    
    # Get ratings of books read by lord_readers
    lord_books_ratings = ratings_data[ratings_data['User-ID'].isin(lord_readers)][['ISBN', 'Book-Rating']]
    lord_books_ratings['Book-Rating'] = lord_books_ratings['ISBN'].apply(book_rating)
    
    # Filter books with sufficient ratings
    count = lord_books_ratings['ISBN'].value_counts()
    count = count[count > 5]
    
    # Compute mean ratings of selected books
    mean_ratings = lord_books_ratings.groupby('ISBN')['Book-Rating'].mean().reset_index(name='MeanRating')
    mean_ratings = mean_ratings[mean_ratings['MeanRating'] > BookIsLiked]
    mean_ratings = mean_ratings[~mean_ratings['ISBN'].isin(filtered_books['ISBN'])]
    
    # Get the data of recommended books
    recommended_books = books_data[books_data['ISBN'].isin(mean_ratings['ISBN'])]
    
    return recommended_books


In [8]:
## defining variables
BookIsLiked = 6 ## what does it mean that I like the book? How high the rating must be?
BookTitle = 'The Fellowship of the Ring'.upper()
BookAuthor = 'TOLKIEN'

In [9]:
%%time
recommendation(BookIsLiked, BookTitle, BookAuthor)